In [8]:
import re
import unicodedata
from collections import Counter, defaultdict
import numpy as np

В реальном проекте корпус — тысячи/миллионы строк. Здесь сделаем маленький корпус, чтобы все выполнялось мгновенно.

In [9]:
corpus = [
    "NLP is fun! Fun, fun, fun...",
    "I love machine learning and natural language processing.",
    "Tokenization splits text into pieces — tokens.",
    "Cats, cat's, and catlike: morphology matters.",
    "Я люблю NLP и обработку естественного языка.",
    "Кошки любят молоко, а кот любит рыбу.",
    "Привет!   Привет!! Привет???",
    "é vs é  — Unicode normalization example.",
]
corpus

['NLP is fun! Fun, fun, fun...',
 'I love machine learning and natural language processing.',
 'Tokenization splits text into pieces — tokens.',
 "Cats, cat's, and catlike: morphology matters.",
 'Я люблю NLP и обработку естественного языка.',
 'Кошки любят молоко, а кот любит рыбу.',
 'Привет!   Привет!! Привет???',
 'é vs é  — Unicode normalization example.']

**Идея:** модель не понимает строки → мы превращаем текст в *последовательность дискретных единиц* (токенов), а затем в числа.

Типичные шаги:
- **Unicode-нормализация** (NFC/NFKC)
- нормализация пробелов
- приведение к lower (не всегда!)
- обработка чисел/URL/емодзи (зависит от задачи)
- лемматизация/стемминг (опционально, зависит от модели и языка)

In [10]:
def normalize_unicode(text: str, form: str = "NFC") -> str:
    return unicodedata.normalize(form, text)

def normalize_spaces(text: str) -> str:
    return re.sub(r"\s+", " ", text).strip()

def basic_preprocess(text: str) -> str:
    text = normalize_unicode(text, "NFC")
    text = normalize_spaces(text)
    return text

for t in corpus[-2:]:
    print("RAW: ", repr(t))
    print("NFC: ", repr(basic_preprocess(t)))
    print()

RAW:  'Привет!   Привет!! Привет???'
NFC:  'Привет! Привет!! Привет???'

RAW:  'é vs é  — Unicode normalization example.'
NFC:  'é vs é — Unicode normalization example.'



## Стемминг VS Лемматизация

**Нормализация**: привести разные варианты записи к одному виду (Unicode, ё/е, кавычки, пробелы).

**Лемматизация**: привести слово к “словарной форме”:
- *cats → cat*
- *кошки → кошка*

Зачем:
- уменьшить разреженность в count-based признаках
- лучше обобщать на формы слова

Но:
- для современных subword-токенизаторов/LLM лемматизация часто **не нужна** и иногда вредна (ломает стиль/смысл).

In [11]:
# Examples: stemming vs lemmatization (English)

# pip install nltk spacy
# python -m spacy download en_core_web_sm

# -------------------- NLTK: stemming --------------------
import nltk
from nltk.stem import PorterStemmer

# If you haven't downloaded NLTK resources yet:
# nltk.download("wordnet")
# nltk.download("omw-1.4")
# nltk.download("averaged_perceptron_tagger_eng")  # new tagger name in recent NLTK
# nltk.download("averaged_perceptron_tagger")      # fallback for older NLTK

stemmer = PorterStemmer()

words = ["running", "runner", "runs", "better", "cars", "studies", "studying", "wolves", "went"]
print("Stemming (Porter):")
for w in words:
    print(f"{w:10s} -> {stemmer.stem(w)}")


from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def to_wordnet_pos(treebank_tag: str):
    """Map Penn Treebank POS tags to WordNet POS tags (rough but useful)."""
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    if treebank_tag.startswith("V"):
        return wordnet.VERB
    if treebank_tag.startswith("N"):
        return wordnet.NOUN
    if treebank_tag.startswith("R"):
        return wordnet.ADV
    return wordnet.NOUN  # default

print("\nLemmatization (WordNet) WITHOUT POS (default noun):")
for w in words:
    print(f"{w:10s} -> {lemmatizer.lemmatize(w)}")

print("\nLemmatization (WordNet) WITH POS (better):")
tagged = nltk.pos_tag(words)
for w, tag in tagged:
    wn_pos = to_wordnet_pos(tag)
    print(f"{w:10s} ({tag:4s}) -> {lemmatizer.lemmatize(w, pos=wn_pos)}")

Stemming (Porter):
running    -> run
runner     -> runner
runs       -> run
better     -> better
cars       -> car
studies    -> studi
studying   -> studi
wolves     -> wolv
went       -> went

Lemmatization (WordNet) WITHOUT POS (default noun):
running    -> running
runner     -> runner
runs       -> run
better     -> better
cars       -> car
studies    -> study
studying   -> studying
wolves     -> wolf
went       -> went

Lemmatization (WordNet) WITH POS (better):
running    (VBG ) -> run
runner     (NN  ) -> runner
runs       (VBZ ) -> run
better     (JJR ) -> good
cars       (NNS ) -> car
studies    (NNS ) -> study
studying   (VBG ) -> study
wolves     (NNS ) -> wolf
went       (VBD ) -> go


### Простая токенизация “по словам” (baseline)

In [7]:
token_re = re.compile(r"[\w']+|[^\w\s]", re.UNICODE)

def simple_word_tokenize(text: str):
    text = basic_preprocess(text)
    return token_re.findall(text)

for s in corpus[:3]:
    print(s)
    print(simple_word_tokenize(s))
    print()

NLP is fun! Fun, fun, fun...
['NLP', 'is', 'fun', '!', 'Fun', ',', 'fun', ',', 'fun', '.', '.', '.']

I love machine learning and natural language processing.
['I', 'love', 'machine', 'learning', 'and', 'natural', 'language', 'processing', '.']

Tokenization splits text into pieces — tokens.
['Tokenization', 'splits', 'text', 'into', 'pieces', '—', 'tokens', '.']



### Subword-токенизация: BPE / WordPiece / Unigram (12–14 мин)

**Почему subword:**
- решает проблему OOV (out-of-vocabulary)
- хорошо работает на морфологически богатых языках
- балансирует между “словом” и “символом”

Коротко:
- **BPE**: итеративно сливает самые частые пары токенов.
- **WordPiece**: похож на BPE, но оптимизирует “правдоподобие” (используется в BERT).
- **Unigram**: начинает с большого словаря подслов и удаляет худшие (SentencePiece).

Ниже — обучим игрушечные токенизаторы на нашем корпусе и сравним разбиение.

In [ ]:
# Попробуем использовать HuggingFace tokenizers. Если нет — дадим пояснение.
try:
    from tokenizers import Tokenizer
    from tokenizers.models import BPE, WordPiece, Unigram
    from tokenizers.trainers import BpeTrainer, WordPieceTrainer, UnigramTrainer
    from tokenizers.pre_tokenizers import Whitespace
    from tokenizers.normalizers import NFKC
    TOKENIZERS_OK = True
except Exception as e:
    TOKENIZERS_OK = False
    print("HuggingFace tokenizers not available:", type(e).__name__, "-", e)

TOKENIZERS_OK

In [ ]:
texts = [basic_preprocess(t) for t in corpus]
texts

In [ ]:
def train_bpe(texts, vocab_size=80):
    tok = Tokenizer(BPE(unk_token="[UNK]"))
    tok.normalizer = NFKC()
    tok.pre_tokenizer = Whitespace()
    trainer = BpeTrainer(vocab_size=vocab_size, special_tokens=["[UNK]","[PAD]","[CLS]","[SEP]","[MASK]"])
    tok.train_from_iterator(texts, trainer=trainer)
    return tok

def train_wordpiece(texts, vocab_size=80):
    tok = Tokenizer(WordPiece(unk_token="[UNK]"))
    tok.normalizer = NFKC()
    tok.pre_tokenizer = Whitespace()
    trainer = WordPieceTrainer(vocab_size=vocab_size, special_tokens=["[UNK]","[PAD]","[CLS]","[SEP]","[MASK]"])
    tok.train_from_iterator(texts, trainer=trainer)
    return tok

def train_unigram(texts, vocab_size=80):
    tok = Tokenizer(Unigram())
    tok.normalizer = NFKC()
    tok.pre_tokenizer = Whitespace()
    trainer = UnigramTrainer(vocab_size=vocab_size, special_tokens=["[UNK]","[PAD]","[CLS]","[SEP]","[MASK]"])
    tok.train_from_iterator(texts, trainer=trainer)
    return tok

if TOKENIZERS_OK:
    bpe_tok = train_bpe(texts, vocab_size=80)
    wp_tok  = train_wordpiece(texts, vocab_size=80)
    uni_tok = train_unigram(texts, vocab_size=80)

In [ ]:
sample = "Кошки любят молоко, а кот любит рыбу. Tokenization is fun!"
sample = basic_preprocess(sample)

if TOKENIZERS_OK:
    print("SAMPLE:", sample)
    print("\nBPE:", bpe_tok.encode(sample).tokens)
    print("\nWordPiece:", wp_tok.encode(sample).tokens)
    print("\nUnigram:", uni_tok.encode(sample).tokens)
else:
    print("Skip: tokenizers library not available.")

**Что обсудить по результатам:**
- где появляется много мелких кусочков?
- как ведут себя русские окончания?
- как отличаются “маркер продолжения слова” (в WordPiece часто `##...`) — зависит от реализации/словаря.

## 4) Count-based вектора: Bag-of-Words и TF-IDF (8–10 мин)

**Bag-of-Words (BoW)**: считаем, сколько раз токены встречаются в документе.  
**TF-IDF**: downweight частые слова, upweight специфические.

Плюсы:
- быстро, просто, часто работает “удивительно хорошо” на классификации.

Минусы:
- теряется порядок слов (мешок слов)
- огромная разреженность
- слабая переносимость на новые домены
- плохо с семантикой (синонимы, перефразирование)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

toy_docs = [
    "I love this movie, it is fantastic and fun",
    "This film was terrible, boring and bad",
    "What a great and wonderful experience",
    "Awful plot, bad acting, I hate it",
    "Fantastic acting and great story",
    "Boring movie, not good",
]
toy_y = np.array([1,0,1,0,1,0])  # 1=positive, 0=negative

bow = CountVectorizer(lowercase=True)
X_bow = bow.fit_transform(toy_docs)

tfidf = TfidfVectorizer(lowercase=True)
X_tfidf = tfidf.fit_transform(toy_docs)

print("BoW shape:", X_bow.shape, "nnz:", X_bow.nnz)
print("TF-IDF shape:", X_tfidf.shape, "nnz:", X_tfidf.nnz)
print("\nVocabulary:", list(bow.vocabulary_.keys())[:10], "...")

In [ ]:
# Посмотрим топ-слова по TF-IDF в одном документе
doc_id = 0
row = X_tfidf[doc_id].toarray().ravel()
top = row.argsort()[::-1][:8]
inv_vocab = {i:w for w,i in tfidf.vocabulary_.items()}
[(inv_vocab[i], row[i]) for i in top if row[i] > 0]

### Ограничения count-based подходов (что проговорить)

1) **Нет порядка**: “dog bites man” vs “man bites dog” → одинаково (в BoW).  
2) **Синонимы**: “great” и “excellent” — разные координаты.  
3) **Разреженность**: словарь растёт, память/время растут.  
4) **Доменные сдвиги**: новые слова/жанры → деградация.  
5) **Морфология**: без лемматизации “кошки/кошку/кошкой” раздувают словарь.

## 5) Эмбеддинги: Word2Vec, GloVe, “плотные” представления (7–9 мин)

**Эмбеддинг**: отображение токена в плотный вектор `R^d`, где близость ≈ семантическая близость.

- **Word2Vec (CBOW/Skip-gram)**: предсказываем контекст по слову или слово по контексту.
- **GloVe**: использует глобальные матрицы совместной встречаемости (co-occurrence) и факторизацию с весами.

Здесь:
- покажем идею на мини-примере
- (опционально) обучим маленький Word2Vec, если доступен `gensim`

In [ ]:
# Идея: если два слова часто встречаются в похожих контекстах — их вектора должны быть близки.

contexts = [
    ("king", ["man","royal","crown"]),
    ("queen", ["woman","royal","crown"]),
    ("apple", ["fruit","sweet","tree"]),
    ("orange", ["fruit","sweet","citrus"]),
]
contexts

In [ ]:
def try_train_word2vec(sentences):
    try:
        from gensim.models import Word2Vec
        model = Word2Vec(sentences=sentences, vector_size=50, window=3, min_count=1, sg=1, epochs=200, workers=1)
        return model
    except Exception as e:
        print("gensim Word2Vec not available:", type(e).__name__, "-", e)
        return None

# сделаем "предложения" из нашего мини-корпуса
sentences = [simple_word_tokenize(t.lower()) for t in corpus]
w2v = try_train_word2vec(sentences)

if w2v is not None:
    print(w2v.wv.most_similar("fun", topn=5))

**Обсуждение:**
- На маленьком корпусе качества ждать не надо — цель увидеть *механику*.
- На большом корпусе Word2Vec/GloVe дают полезные “геометрические” свойства.

**Ограничения классических word embeddings:**
- один вектор на слово → полисемия (“bank” как берег/банк)
- контекст не учитывается (в отличие от BERT/LLM)

## 6) Базовая классификация текста (8–10 мин)

Собираем простой pipeline:
- текст → TF-IDF
- классификатор: Logistic Regression / Linear SVM / Naive Bayes

Это “рабочая лошадка” для задач:
- sentiment
- topic classification
- spam detection

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(toy_docs, toy_y, test_size=0.33, random_state=42, stratify=toy_y)

clf = Pipeline([
    ("tfidf", TfidfVectorizer(lowercase=True, ngram_range=(1,2))),
    ("lr", LogisticRegression(max_iter=1000))
])

clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print("Test docs:", X_test)
print("Pred:", pred, "True:", y_test)
print()
print(classification_report(y_test, pred, digits=3))
print("Confusion matrix:\n", confusion_matrix(y_test, pred))

### Интерпретация: какие признаки важны?

Для линейных моделей на TF-IDF можно посмотреть веса по словам/нграммам.

In [ ]:
vec = clf.named_steps["tfidf"]
lr = clf.named_steps["lr"]

feature_names = np.array(vec.get_feature_names_out())
coef = lr.coef_[0]  # positive class
top_pos = coef.argsort()[::-1][:10]
top_neg = coef.argsort()[:10]

print("Top positive features:")
for i in top_pos:
    print(f"{feature_names[i]:<20} {coef[i]:.3f}")

print("\nTop negative features:")
for i in top_neg:
    print(f"{feature_names[i]:<20} {coef[i]:.3f}")

## 7) Итоги (1–2 мин)

- Текст → токены → индексы → вектора → модель
- Предобработка помогает, но может и вредить (зависит от задачи)
- Subword-токенизация (BPE/WordPiece/Unigram) решает OOV и помогает морфологии
- Count-based (BoW/TF-IDF) — сильный baseline, но без семантики и порядка
- Эмбеддинги (Word2Vec/GloVe) дают “плотную” семантику, но контекст ограничен
- Базовая классификация = TF-IDF + линейная модель — отличный стартовый baseline

### Домашнее/практика (если останется 3–5 минут)

1) Добавьте в TF-IDF `min_df` и `max_df`: как меняется словарь?  
2) Попробуйте `MultinomialNB` и `LinearSVC` вместо логрегрессии.  
3) Сравните токенизацию (word vs subword) для пары русских словоформ.